### Madden Ultimate Team (MUT) Auction House Analyzer

A set in madden is where you buy n specific cards to complete the set and recieve a more valuable player in return.
Given an input of set players, ids, and the reward allows you to find arbitrage in the MUT Auction House.

Currently:
- Currently running two parallel arrays of set players and their ids
- Keep track of target player and target_id 
- Use selenium to open webdriver Firefox
- Manually log into 'Muthead' (Must be logged in to refresh auction prices)
- Selenium refreshes all of the auction prices of relevant players page by page
- Use beautifulsoup to scrape from three tables: Current Auctions, Completed Sales, and 'Stats for Nerds'
- Using the minimum BuyNow price for each player, the price to complete the set is calculated
- Using this, an average of last n sales of player and taking into account the 10% tax fee on the Auction House a conditional is triggered to pursue the set based on expected profit

Current Pitfalls:
- Need to manually login
- Poor flow
- Likely needs proper use of .sleep() 

Planned:
- store data, pickle?
- to also use for non-set (rare) players


In [103]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import requests
import warnings; warnings.simplefilter('ignore')  
import time

## methods

In [104]:
def retrieve_prices(auction_table):    # method that returns a dataframe of current auctions
    
    #print (soup.title.text) # print player page title
    
    rows = auction_table.find_all('tr')
    num_rows = min ( len(rows), 5 )
    #title row
    row = rows[0]
    td_tags = row.find_all('td')
    th_tags = row.find_all('th') 

    titles = []

    for tag in th_tags:
        titles.append(str(tag)[4:-5]) #remove tags
    
    num_cols = len(titles)

    new_table = pd.DataFrame(columns=titles, index=range(0,num_rows-1))

    for i in range(1,num_rows):
        row = rows[i]

        td_tags = row.find_all('td')

        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
    
        #time remaining
        try:
            time_rem = values[0][58:67].replace("<", "").replace(">", "")
        except IndexError: 
            time_rem = '0'
        try:
            num_bids = values[1]
        except IndexError:
            num_bids = '0'
        try:
            curr_bid = values[2]
        except IndexError:
            curr_bid = '0'
        try:
            buy_now = values[3]
        except IndexError:
            buy_now = '0'
            
        nums = [time_rem, num_bids, curr_bid, buy_now]
        j=0
    
        for title in titles:
            new_table[title][i-1] = nums[j]
            j += 1
    
    return new_table

In [105]:
def retrieve_stats(hist_table):  # method to return dataframe of stats for last 100 sales such as quartile prices, min, avg
    
    rows = hist_table.find_all('tr')

    new_table2 = pd.DataFrame(columns=['Q', 'Price'], index=[0,1,2,3,4,5,6])
    
    for i in range(0,7): #first row of prices
        row = rows[i]
        td_tags = row.find_all('td')
        
        values = []

        for val in td_tags:
            values.append(str(val))
 
        #time remaining
        ca = values[0]
        cb = values[1]
    
        nums = [ca,cb]
        new_table2['Q'][i] = ca
        new_table2['Price'][i] = cb
    
    return new_table2

In [106]:
def retrieve_sales(sales_table):    # method to return dataframe of last n completed sales ( <= 10?)
    
    rows = sales_table.find_all('tr')
    num_rows = min ( len(rows), 10 )
    
    row = rows[0] #title row
    td_tags = row.find_all('td')
    th_tags = row.find_all('th') 
    
    titles = []

    for tag in th_tags:
        titles.append(str(tag)[4:-5]) #remove tags
    
    num_cols = len(titles)
    new_table3 = pd.DataFrame(columns=titles, index=range(0,num_rows-1))
   
    for i in range(1,num_rows):
        row = rows[i]

        td_tags = row.find_all('td')

        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
            
        time_sold = values[0][119:140].replace("<", "").replace(">", "")
        price_sold = values[1].replace(".", "").replace(",", "")
        
        nums = [time_sold, price_sold] #print(nums) # debug
        j=0
    
        for title in titles:
            new_table3[title][i-1] = nums[j]
            j += 1
            
    new_table3['Price'] = pd.to_numeric( new_table3['Price'] )
    return new_table3

## price refresh

In [43]:
# RUN THIS

browser = webdriver.Firefox()

URL = 'https://www.muthead.com/twitch-login?returnUrl=%2f'

browser.get(URL)

#manually log in

# data frame to store scraped data
data = pd.DataFrame(columns=['Name', 'Min Price', 'Average', 'Date'], index=[0])

# start here
PLAYERS WE WANT TO SEARCH ARE DIVIDED INTO TWO LISTS OF NAMES AND PLAYERIDS


In [107]:
player_df = pd.read_csv('player_investments.csv', names=['Program', 'Player', 'id'])
player_df['count']=1
player_df['count'][18]=3 # fb incognito
player_df['count'][17]=3 # ce telvin
player_df['count'][20]=2 # landry harvest
player_df['count'][28]=2 # fb hicks
player_df['count'][47]=3 # ce diggs
player_df['count'][43]=2 # ce burf
player_df['count'][29]=2 # fb berry
player_df['count'][48]=2 # ce lee
player_df['count'][36]=2 # ce drc
player_df['count'][37]=2 # ce dj
player_df['curr_min']=0
player_df['avg']=0
player_df['q1']=0
player_df

,Program,Player,id,count,curr_min,avg,q1
0,PO,Kelce,28552,1,0,0,0
1,TOTW,Kelce,24053,1,0,0,0
2,CE,Kelce,1018000890,1,0,0,0
3,FB,Luck,25025,1,0,0,0
4,CE,Luck,1018011209,1,0,0,0
5,VET,Wilkerson,25423,1,0,0,0
6,TOTW,Wilkerson,24112,1,0,0,0
7,CE,Wilkerson,1018010699,1,0,0,0
8,PO,Houston,28523,1,0,0,0
9,CE,Houston,1018010703,1,0,0,0


In [108]:

player_names = player_df['Player']
player_ids = player_df['id']


In [109]:
len(player_ids)

52

### use only when necessary, refreshes all player data from site

In [44]:
import time

link_start = 'https://www.muthead.com/18/players/prices/'
link_end = '/playstation-4/'
link_end_ref = '/playstation-4/refresh/'
start = time.time()
print("timestart")

for k in range(0, len(player_ids)):
    linky = link_start + player_ids[k] + link_end_ref
    browser.get(linky)
    time.sleep(1)
    #browser.quit()  # uncomment to close browser but will log you out of site

end = time.time()
print('Time in seconds for completion:\t', end - start)

timestart
Time in seconds for completion:	 25.622355937957764


In [110]:
import time

def start_up():
    
    sdf = player_df
    
    start = time.time() # INITIALIZE TIMER FOR COMPUTING TIME
    print("timestart")

    link_start = 'https://www.muthead.com/18/players/prices/' # LINK PREFIX/POSTFIX
    link_end = '/playstation-4/'

    min_prices = [] # LIST FOR LOWEST BUYITNOW PRICE PER PLAYER
    N = len(player_ids)

    for i in range(0,N):
    
        print('startup.... i= ', i)
        
        mut_url = link_start + str(player_ids[i]) + link_end
        print(mut_url)
        try:
            r = urllib.request.urlopen(mut_url).read()
    
            soup = BeautifulSoup(r, 'lxml')
    
            tables = soup.select('table')
    
            print('\n= = = = = ', player_names[i], '= = = = = ')
    
            print('retrieving prices')
            df = retrieve_prices(tables[0])
            time.sleep(2) 
    
            print('retrieving stats')
            df2 = retrieve_stats(tables[2])
            time.sleep(2)
            
            df3 = retrieve_sales(tables[1])
            
            current_min = df['Buy Now Price'][0].replace(",", "")
            lower_q = df2['Price'][5].replace(",", "")
            last_x_mean = df3['Price'].mean()
    
            print()
            current_min = df['Buy Now Price'][0].replace(",", "")
            lower_q = df2['Price'][5].replace(",", "")
            lower_q = re.sub("[^0-9]", "", str(lower_q))
    # potential deal
    
    #if ( int(current_min) < int(lower_q) ):     # add back , curr out from string . to float err
        #print ('======DEAL======')
        
            print ('Current Min:\t', current_min)
            #print ('Avg Price:\t', avg)
            print ('Q1 Price:\t', lower_q)
            min_prices.append(current_min)
            
            
            player_df['curr_min'][i] = current_min  # # #
            player_df['avg'][i] = last_x_mean
            player_df['q1'][i] = lower_q 
            
            temp = time.time()
            print('Time elapsed:\t', temp - start)
    

        except requests.ConnectionError as e:
            print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
            print(str(e))
        except requests.Timeout as e:
            print("OOPS!! Timeout Error")
            print(str(e))
        except requests.RequestException as e:
            print("OOPS!! General Error")
            print(str(e))
        except KeyboardInterrupt:
            print("Someone closed the program")
        
    end = time.time()
    print('Time in seconds for completion:\t', end - start)
    return sdf

In [111]:
player_df['id'][15]

1018028387

In [112]:
inv_df = start_up()
backup_df = inv_df
#put_together()

timestart
startup.... i=  0
https://www.muthead.com/18/players/prices/28552/playstation-4/

= = = = =  Kelce = = = = = 
retrieving prices
retrieving stats

Current Min:	 88000
Q1 Price:	 69750
Time elapsed:	 5.70541787147522
startup.... i=  1
https://www.muthead.com/18/players/prices/24053/playstation-4/

= = = = =   Kelce = = = = = 
retrieving prices
retrieving stats

Current Min:	 167000
Q1 Price:	 102500
Time elapsed:	 11.884022951126099
startup.... i=  2
https://www.muthead.com/18/players/prices/1018000890/playstation-4/

= = = = =   Kelce = = = = = 
retrieving prices
retrieving stats

Current Min:	 50000
Q1 Price:	 30000
Time elapsed:	 17.705111980438232
startup.... i=  3
https://www.muthead.com/18/players/prices/25025/playstation-4/

= = = = =   Luck = = = = = 
retrieving prices
retrieving stats

Current Min:	 169000
Q1 Price:	 142000
Time elapsed:	 23.748008012771606
startup.... i=  4
https://www.muthead.com/18/players/prices/1018011209/playstation-4/

= = = = =   Luck = = = = =


= = = = =   DRC = = = = = 
retrieving prices
retrieving stats

Current Min:	 55000
Q1 Price:	 34250
Time elapsed:	 214.11093187332153
startup.... i=  37
https://www.muthead.com/18/players/prices/1018002406/playstation-4/

= = = = =   David Johnson = = = = = 
retrieving prices
retrieving stats

Current Min:	 53000
Q1 Price:	 41000
Time elapsed:	 219.86055707931519
startup.... i=  38
https://www.muthead.com/18/players/prices/27602/playstation-4/

= = = = =   Shaq Thompson = = = = = 
retrieving prices
retrieving stats

Current Min:	 75000
Q1 Price:	 33125
Time elapsed:	 226.1800298690796
startup.... i=  39
https://www.muthead.com/18/players/prices/1018002562/playstation-4/

= = = = =   Shaq Thompson = = = = = 
retrieving prices
retrieving stats

Current Min:	 0
Q1 Price:	 7000
Time elapsed:	 232.18567872047424
startup.... i=  40
https://www.muthead.com/18/players/prices/1018028824/playstation-4/

= = = = =   Ramon Foster = = = = = 
retrieving prices
retrieving stats

Current Min:	 49500


In [132]:
#pdf = backup_df
pdf = df

pdf['val_by_curr'] = 0
pdf['val_by_avg'] = 0

for _ in pdf.index:
    if (pdf['curr_min'][_] == 0):
        pdf['curr_min'][_] = pdf['avg'][_]

pdf['val_by_curr'] = pdf['curr_min'] * pdf['count'] 
pdf['val_by_avg'] = pdf['avg'] * pdf['count'] 


In [135]:
print( 'value by current min price: ', pdf['val_by_curr'].sum() )
print( 'value by avg price for last ~10 sales: ', pdf['val_by_avg'].sum() )

pdf.to_csv('m23_mut_investments.csv')

value by current min price:  4740575
value by avg price for last ~10 sales:  3728125


In [114]:
inv_df.to_csv('invests.csv')

In [138]:
pdf.head()

,Unnamed: 0,Program,Player,id,count,curr_min,avg,q1,val_by_curr,val_by_avg
0,0,PO,Kelce,28552,1,88000,74472,69750,88000,74472
1,1,TOTW,Kelce,24053,1,167000,134444,102500,167000,134444
2,2,CE,Kelce,1018000890,1,50000,33944,30000,50000,33944
3,3,FB,Luck,25025,1,169000,147444,142000,169000,147444
4,4,CE,Luck,1018011209,1,20000,19083,15250,20000,19083


In [171]:
import numpy as np
from datetime import datetime

pdf2 = pdf

pdf2['date'] = str(datetime.now())[5:-10]
pdf2['date'] = pdf2['date']

piv = pd.pivot_table(pdf2,index=["Player", 'Program'], columns=['date'], values=['count', 'avg', 'val_by_avg', 'curr_min'],aggfunc=np.sum)
piv

avg       count    curr_min  val_by_avg
date                     03-23 22:19 03-23 22:19 03-23 22:19 03-23 22:19
Player           Program                                                
  Burnett        VET           78416           1      100000       78416
  Incognito      CE            24316           1       35000       24316
 Ali Marpet      CE            15944           1       14750       15944
                 TOTW          86611           1       89500       86611
 Burnett         CE            10033           1        9500       10033
 DRC             CE            45416           2       55000       90832
 David Johnson   CE            54472           2       53000      108944
 Demarco Murray  CE            13938           1       13500       13938
 Dez Bryant      CE            22588           1       30000       22588
                 MOTIV         87055           1      109000       87055
 Drew Brees      CE            23294           1       44750       23294
                 HERO         185000           1      263000      185000
 Earl Thomas III CE            52277           1       65000       52277
                 TOTW         249000           1      287000      249000
 Eric Berry      FB            52611           2       52611      105222
 Everson Griffen CE             8761           1       13250        8761
                 TOTW         119222           1      120000      119222
 Houston         CE            22238           1       29000       22238
                 PO            86305           1      138000       86305
 Incognito       FB            52194           3       60000      156582
 Jarvis Landry   CE            27333           1       37500       27333
                 HARV         129555           2      129555      259110
 Jerrell Freeman CE            21338           1       66500       21338
 Jordan Hicks    FB            51083           2       39500      102166
 Jordan Howard   CE            10516           1       10750       10516
 Jurrell Casey   CE             7788           1       13500        7788
                 MOTIV         44888           1       49750       44888
 Kelce           CE            33944           1       50000       33944
                 TOTW         134444           1      167000      134444
 Luck            CE            19083           1       20000       19083
                 FB           147444           1      169000      147444
 Marcell Dareus  CE           117516           1        8900      117516
 Marshal Yanda   CE            55194           1       57500       55194
 Ramon Foster    CE            30288           1       49500       30288
 Richard Sherman CE           142222           1      135000      142222
                 MF           122666           1      440000      122666
 Sean Lee        CE            33361           2       45000       66722
 Shaq Thompson   CE             7500           1        7500        7500
                 FRZ           42416           1       75000       42416
 Stefon Diggs    CE            10133           3       10133       30399
                 TOTW          81666           1      149000       81666
 Talib           CE            33083           1       69500       33083
                 HERO         128500           1      114000      128500
 Telvin Smith    CE             8077           3        7000       24231
                 TOTW         187388           1      267000      187388
 Vontaze Burfict CE            14155           2       16500       28310
 Wilkerson       CE             5355           1      100000        5355
                 TOTW          34694           1       43750       34694
                 VET           84444           1       84444       84444
 Zach Strief     CE            17705           1       10500       17705
                 VET           58722           1       82000       58722
Kelce            PO            74472           1       88000       74472

In [168]:
pdf2 = pdf2.drop('Unnamed: 0', axis=1)


In [170]:
pdf2.to_csv('m23_mutinvest_dated.csv')

In [73]:
def put_together():
    start = time.time()

    summ = 0
    for i in range(0,len(player_names)):
        summ+= int(min_prices[i])
    
    print('Cost to Build:\t', summ)

    mut_url = link_start + target_id + link_end
    r = urllib.request.urlopen(mut_url).read()
    soup = BeautifulSoup(r,"lxml")
    tables = soup.select('table')
    time.sleep(1) 

    print('retrieving prices')
    df = retrieve_prices(tables[0])
    time.sleep(1) 

    print('retrieving stats')
    df2 = retrieve_stats(tables[2])
    print()

    df3 = retrieve_sales(tables[1])

    current_min = df['Buy Now Price'][0].replace(",", "")
    lower_q = df2['Price'][5].replace(",", "")
    last_x_mean = df3['Price'].mean()
       
    cur_min = float (current_min )
    after_tax = cur_min*0.9

    if ( after_tax > summ):
        print('==+++===== BUY! BUY! BUY! =====+++==\n')
    else:
        print('X X X X X X NOT WORTH IT! X X X X X X\n')
    
    for i in range(0, len(player_names)):
        print( player_names[i] + '\t' + min_prices[i] )

    print ('\nCost to build:\t\t', summ)
    print ('Lowest ' + target +':\t', int(cur_min))
    print ('After Tax :\t\t', int(after_tax))
    print ('Last x mean:\t\t', int(last_x_mean) )
    print('Potential profit:\t', int(after_tax-summ))

    end = time.time()
    timey = end - start
    timey = str(round(timey, 2))

    print('\nTime in seconds for completion:\t', timey)


In [44]:
# todo: code to save into dataframe (long term)
#min_prices

In [173]:
pdf2.head()

,Unnamed: 0,Program,Player,id,count,curr_min,avg,q1,val_by_curr,val_by_avg,date
0,0,PO,Kelce,28552,1,88000,74472,69750,88000,74472,03-23 22:19
1,1,TOTW,Kelce,24053,1,167000,134444,102500,167000,134444,03-23 22:19
2,2,CE,Kelce,1018000890,1,50000,33944,30000,50000,33944,03-23 22:19
3,3,FB,Luck,25025,1,169000,147444,142000,169000,147444,03-23 22:19
4,4,CE,Luck,1018011209,1,20000,19083,15250,20000,19083,03-23 22:19


In [64]:

#sdf = pd.DataFrame(columns=['player', 'id', 'min', 'avg', 'q1', 'date'], index=range(0,len(player_ids)))
sdf = player_df
#sdf.columns = ['Program', 'Player', 'id', 'count', 'curr_min', 'avg', 'q1', 'date']
sdf

,Program,Player,id,count,curr_min
0,PO,Kelce,28552,1,45000
1,TOTW,Kelce,24053,1,45000
2,CE,Kelce,1018000890,1,45000
3,FB,Luck,25025,1,45000
4,CE,Luck,1018011209,1,45000
5,VET,Wilkerson,25423,1,45000
6,TOTW,Wilkerson,24112,1,45000
7,CE,Wilkerson,1018010699,1,45000
8,PO,Houston,28523,1,45000
9,CE,Houston,1018010703,1,45000


###### Matthew Johnson, March 21, 2018